# Palm vein recognition


Ici, vous allez trouver tous les étapes nécessaires pour développer un modèle de CNN pour la reconnaissance de veines palmaires. 

Ce projet a été développé par : Kenan GONNOT, Lorenzo MARQUES et Fayçal MERZOUK.  

**L’objectif** est de créer un modèle de veines verification : «Est-ce la bonne personne ? »

Vous pouvez retrouver ce code sur [GitHub](https://github.com/kenanGonnot/cnn_palmar_veins). 



In [ ]:
## Table of Contents

- [1 - Imports](#1)
- [2 - Load images - Train(70%) Test(15%) Val(15%)](#2)
- [4 - Create model CNN](#6)

## Imports

In [ ]:
import gc
import glob
import os
import sys
import cv2
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from random import randrange
from tqdm import tqdm, tnrange
from tqdm.notebook import tqdm_notebook

import warnings

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

from keras.optimizer_v2.gradient_descent import SGD
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Dense, MaxPooling2D, Flatten, Conv2D, Lambda, Dropout, LeakyReLU, BatchNormalization, Activation, AveragePooling2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

from mlxtend.evaluate import accuracy

from tensorflow.keras.optimizers import Adam,RMSprop,SGD, Nadam
#from tensorflow.keras.utils import to_categorical
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # ignore tensorflow warnings
warnings.filterwarnings("ignore")

## Load images - Train(70%) Test(15%) Val(15%)

In [ ]:
path_data = "data/data_palm_vein/NIR"
weight_path='saved_model/resnet50v2TL_20epochs_32batch.h5'

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    path_data,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=1007,
    validation_split=0.3,
    subset="training",
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    path_data,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=1007,
    validation_split=0.15,
    subset="validation",
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    path_data,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=1007,
    validation_split=0.15,
    subset="validation",
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
data = tf.keras.preprocessing.image_dataset_from_directory(
    path_data,
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=1007,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

## Creation of the model - ResNet50 with Transfer Learning

In [ ]:
from keras.applications.resnet_v2 import ResNet50V2

def resnet_model_tf(input_shape=(128, 128, 3), nombre_classes=500):
    resnet = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)
    resnet.tbatch_sizenable = False
    model = Sequential()
    model.add(resnet)
    model.add(Flatten())
    model.add(Dense(4096, activation='LeakyReLU'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='LeakyReLU'))
    model.add(Dropout(0.5))
    model.add(Dense(nombre_classes, activation='softmax'))
    
    print(model.summary())
    model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.7, 
                                            min_lr=0.00000000001)
    return model, learning_rate_reduction

In [ ]:
model, learning_rate_reduction = resnet_model_tf()

In [ ]:
# Loads the weights
model.load_weights(weight_path)

In [ ]:
loss, acc = model.evaluate(test_ds, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

In [ ]:
X, y = list(data)[50]

In [ ]:
y_pred = model.predict(X)

In [ ]:
np.argmax(y)

In [ ]:
np.argmax(y_pred)

In [ ]:
data.take(1)

In [ ]:
plt.figure(figsize=(22, 24))
lass_names = data.class_names
for images, labels in data.take(1):
    for i in range(32):
        ax = plt.subplot(6, 6, i + 1)
        y_pred = model.predict(images)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title("Real Identity is {}\nPredict Identity is {}".format(np.argmax(labels[i]), y_pred[i].argmax()))
        plt.axis("off")